In [1]:
import os
import numpy as np
#assumes labelled data ra stored into a positive and negative folder
#returns two lists one with the text per file and another with the corresponding class 
path_baptiste = "/home/baptiste/Documents/data/train"


def loadLabeled(path):

	rootdirPOS =path+'/pos'
	rootdirNEG =path+'/neg'
	data=[]
	Class=[]
	count=0
	for subdir, dirs, files in os.walk(rootdirPOS):
		
		for file in files:
			with open(rootdirPOS+"/"+file, 'r') as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc1=np.ones(len(data))
	for subdir, dirs, files in os.walk(rootdirNEG):
		
		for file in files:
			with open(rootdirNEG+"/"+file, 'r') as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
	tmpc0=np.zeros(len(data)-len(tmpc1))
	Class=np.concatenate((tmpc1,tmpc0),axis=0)
	return data,Class
#loads unlabelled data	
#returns two lists
#one with the data per file and another with the respective filenames (without the file extension)
def loadUknown(path):
	rootdir=path
	data=[]
	names=[]
	for subdir, dirs, files in os.walk(rootdir):
		for file in files:
			with open(rootdir+"/"+file, 'r') as content_file:
				content = content_file.read() #assume that there are NO "new line characters"
				data.append(content)
				names.append(file.split(".")[0])
	return data,names

Load the data

Creating new features 
- number of !!! 
- sentence length (before and after removing anything)
- Find if they give a grade, smiley.
- Find the grouped ponctuation points (... has different meaning, smileys)
- Find N-grams where it may start with a CAPITAL (As for the movie names and actor's names)
- Find a list of the movie so as it matches some in the reviews : www.imdb.com

Removing ponctuation

Removing stop words

Stemming algorithm

Other feature creation (unigrams, N-grams) compute de frequency of t in document d: $f_{t,d}$

find the frequency of a N-gram in the documents $df_t$

tf idf to create the feature matrix

Create 5 csv : train_train.csv, train_test.csv, y_train_train.csv, y_train_test.csv, test.csv